<h1>Dask Delayed - Lazy execution scheduler</h2>

Set up session by importing from dask.distributed and dask delayed

In [1]:
import sys
import os
from dask.distributed import Client
from dask import delayed

Create a cluster, or use default multi-threaded cluster

In [ ]:
client = Client() #This creates a local (pc) multi-process cluster

In [ ]:
client

Show dashboard address (default is localhost:8787)

In [ ]:
client.scheduler_info()['services']

Getting to the dashboard can be a little tricky due to network restrictions.  
The easiest way is run code and dashboard on your laptop.  
If running code on GML server, ssh into the same server in a new terminal window
and create a local redirect:

    ssh nimbus3 -L 8787:localhost:8787
    
and then open a browser window and goto this address:

    http://localhost:8787

Create some functions that simulate work

In [ ]:
from time import sleep
def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

Run

In [ ]:
%%time
# This takes three seconds to run because we call each
# function sequentially, one after the other

x = inc(1)
y = inc(2)
z = add(x, y)

Parallelize by using delayed syntax

In [ ]:
%%time
# This runs immediately, all it does is build a graph

x = delayed(inc)(1)
y = delayed(inc)(2)
z = delayed(add)(x, y)

In [ ]:
z.visualize()

No work has been done in the loop yet until we compute().
Note that that add function can accept delayed inputs (x & y)

In [ ]:
%%time
z.compute()

Similarly with a delayed loop, we can parallelize the operations

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
%%time
# Sequential code

results = []
for x in data:
    y = inc(x)
    results.append(y)
    
total = sum(results)

In [ ]:
%%time
# Parallel code
results=[]
for x in data:
    y=delayed(inc)(x) 
    results.append(y)
total=delayed(sum)(results)
#total=sum(results)
total.visualize()

In [ ]:
%%time
results=total.compute()

In [ ]:
results

Clean up client if we started one.

In [ ]:
client.shutdown()